# A deep dive into DataFrames.jl indexing
# Part 1: indexing in DataFrames.jl by example
### Bogumił Kamiński

`DataFrame` is an object that holds a collection of named columns stored as vectors.

In this tutorial we discuss how to get or set values of these columns.

What are we going to cover:
* `getindex`, a.k.a. `x[...]`
* `setindex!`, a.k.a. `x[...] =`
* `broadcast`, a.k.a. `fun.(x)`
* `broadcast!`, a.k.a. `x .= ...`
* `getproperty`, a.k.a. `x.field` and `x.field .= ...`
* `setproperty`, a.k.a. `x.field = ...`

Indexable types that DataFrames.jl defines:
* `DataFrame`
* `SubDataFrame`
* `DataFrameRow`
* `DataFrameRows`
* `DataFrameColumns`
* `GroupedDataFrame`
* `GroupKeys`
* `GroupKey`
* `StackedVector`
* `RepeatedVector`

### Environment setup

In [1]:
using BenchmarkTools

In [2]:
using CSV

In [3]:
using DataFrames

In [4]:
using Dates

In [5]:
using ShiftedArrays: lag

In [6]:
using Statistics

Let us read-in the data set we will work with.

Make sure you have the required file in the working directory.

The detailed instructions how to get it are in https://github.com/bkamins/JuliaCon2020-DataFrames-Tutorial/blob/master/README.md

In [7]:
fh5 = CSV.read("fh_5yrs.csv", DataFrame)

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


#### Warm up exercises

*Get a short description of columns in our data frame*

(see https://github.com/JuliaData/DataFrames.jl/issues/2269 for a discussion of the design decisions here, feel free to comment there if you have an opinion)

*Get information about exact types of the columns stored in the data frame*

*Get names of columns as strings*

*Get names of columns as `Symbol`s*

## `getindex`

Get a single column as a whole without copying

In [8]:
unique([fh5.date,
        fh5."date",
        fh5[!, 1],
        fh5[!, :date],
        fh5[!, "date"]])

1-element Vector{Vector{Date}}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

In [9]:
unique([getproperty(fh5, :date),
        getproperty(fh5, "date"),
        getindex(fh5, !, 1),
        getindex(fh5, !, :date),
        getindex(fh5,!, "date")])

1-element Vector{Vector{Date}}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

Get a single column as a whole with copying

In [10]:
unique([copy(fh5.date),
        copy(fh5."date"),
        fh5[:, 1],
        fh5[:, :date],
        fh5[:, "date"]])

1-element Vector{Vector{Date}}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

Let us compare the performance of various ways to get a column without copying

In [11]:
@btime $fh5.date
@btime $fh5."date"
@btime $fh5[!, 1]
@btime $fh5[!, :date]
@btime $fh5[!, "date"];

  10.010 ns (0 allocations: 0 bytes)
  28.873 ns (0 allocations: 0 bytes)
  2.300 ns (0 allocations: 0 bytes)
  10.010 ns (0 allocations: 0 bytes)
  31.621 ns (0 allocations: 0 bytes)


`@btime` is from BenchmarkTools.jl package. We use `$` to ensure the time is measured properly.
This is a special syntax specific to `@btime` (like `$` used in string interpolation context).

#### Exercise

Check the same but with copying

Let us check how lookup speed scales with the number of columns:

In [12]:
@time df_tmp = DataFrame(ones(1, 100_000), :auto)

  0.062045 seconds (399.54 k allocations: 27.844 MiB)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,⋯


In [13]:
@btime $df_tmp.x100000
@btime $df_tmp."x100000"
@btime $df_tmp[!, 100000];

  10.310 ns (0 allocations: 0 bytes)
  58.061 ns (0 allocations: 0 bytes)
  2.300 ns (0 allocations: 0 bytes)


<div class="alert alert-block alert-info">
<b>Tip:</b>
    
DataFrames.jl is specifically designed to allow you to handle without huge compilation costs
very wide data frames with heterogeneous column types and changing the schema of the data frame in-place.
(or if you do not want to think if you will not run into these issues)
</div>

Get a single column, but take a subset of rows: you can either make a copy or get a view

In [14]:
fh5[1:2, :date]

2-element Vector{Date}:
 2020-07-02
 2020-07-01

In [15]:
view(fh5, 1:2, :date)

2-element view(::Vector{Date}, 1:2) with eltype Date:
 2020-07-02
 2020-07-01

this is the same as e.g.:

In [16]:
fh5.date[1:2]

2-element Vector{Date}:
 2020-07-02
 2020-07-01

In [17]:
@view fh5.date[1:2]

2-element view(::Vector{Date}, 1:2) with eltype Date:
 2020-07-02
 2020-07-01

you can use `Not` for inverted selection of rows

In [18]:
fh5[Not(3:end), :date]

2-element Vector{Date}:
 2020-07-02
 2020-07-01

Get a single cell in a data frame: you can either get a value or a view

In [19]:
fh5[1, :date]

2020-07-02

In [20]:
fh5[CartesianIndex(1, 1)]

2020-07-02

In [21]:
@view fh5[1, "date"]

0-dimensional view(::Vector{Date}, 1) with eltype Date:
Date("2020-07-02")

In what case you might want to use a view instead of getting a value?

Check what is the consequence of running the following lines:

In [22]:
tmp_cell = view(fh5, 1, :date)

0-dimensional view(::Vector{Date}, 1) with eltype Date:
Date("2020-07-02")

In [23]:
tmp_cell2 = getindex(fh5, 1, :date)

2020-07-02

In [24]:
tmp_cell[] = Date("2222-07-02")

2222-07-02

In [25]:
fh5

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


Revert the change we have just made

In [26]:
tmp_cell[] = tmp_cell2

2020-07-02

In [27]:
fh5

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


To conclude note that with `view` there is no difference between `!` and `:`:

In [28]:
@view fh5[!, 1]

6852038-element view(::Vector{Date}, :) with eltype Date:
 2020-07-02
 2020-07-01
 2020-06-30
 2020-06-29
 2020-06-26
 2020-06-25
 2020-06-24
 2020-06-23
 2020-06-22
 2020-06-19
 2020-06-18
 2020-06-17
 2020-06-16
 ⋮
 2015-01-23
 2015-01-22
 2015-01-21
 2015-01-20
 2015-01-16
 2015-01-14
 2015-01-13
 2015-01-12
 2015-01-09
 2015-01-07
 2015-01-05
 2015-01-02

In [29]:
@view fh5[:, 1]

6852038-element view(::Vector{Date}, :) with eltype Date:
 2020-07-02
 2020-07-01
 2020-06-30
 2020-06-29
 2020-06-26
 2020-06-25
 2020-06-24
 2020-06-23
 2020-06-22
 2020-06-19
 2020-06-18
 2020-06-17
 2020-06-16
 ⋮
 2015-01-23
 2015-01-22
 2015-01-21
 2015-01-20
 2015-01-16
 2015-01-14
 2015-01-13
 2015-01-12
 2015-01-09
 2015-01-07
 2015-01-05
 2015-01-02

#### Exercise

Why is it useful to support `@view` both for `!` and `:`?

<div class="alert alert-block alert-info">
<b>Tip:</b>
    
passing a single column as an integer, `Symbol` or string drops one dimension of
a data frame and allows you to select or subset a column from it (depending on the row selector you choose)
</div>

Multiple column selection options are:
* a vector of `Symbol` (does not have to be a subtype of `AbstractVector{Symbol}`, e.g. `Any[:date]`);
* a vector of `AbstractString` (does not have to be a subtype of `AbstractVector{<:AbstractString}`, e.g. `Any["date"]`);
* a vector of `Integer` other than `Bool` (does not have to be a subtype of `AbstractVector{<:Integer}`, e.g. `Any[1]`);
* a vector of `Bool` that has to be a subtype of `AbstractVector{Bool}`;
* a regular expression, which gets expanded to a vector of matching column names;
* a `Not` expression;
* an `All` or `Between` expression;
* a `Cols` expression;
* a colon literal `:`.

The type of the result depends on the row selecor:
* if it is a single row you get a `DataFrameRow` (a dimension is dropped)
* if it is a collection of rows you get a data frame

Single row selection is always a view that is `DataFrameRow`:

In [30]:
fh5[1, [:date]]

Row,date
,Date
1,2020-07-02


In [31]:
@view fh5[1, [:date]]

Row,date
,Date
1,2020-07-02


Note that `DataFrameRow` is one-dimensional (as usual - single value indexing drops a dimension). You can think of it as a mutable `NamedTuple`.

Making `fg5[1, [:date]]` to be copying would incur overhead that was considered to be not justified by typical use cases of this syntax. (but if you disagree please open an issue on GitHub)

Multiple row selection is a `DataFrame` for `getindex`:

In [32]:
fh5[1:2, 1:2]

Row,date,volume
,Date,Int64
1,2020-07-02,257500
2,2020-07-01,468100


This does not copy columns and is fast

In [33]:
df_tmp = fh5[!, 1:2]

Row,date,volume
,Date,Int64
1,2020-07-02,257500
2,2020-07-01,468100
3,2020-06-30,319100
4,2020-06-29,405500
5,2020-06-26,335100
6,2020-06-25,246800
7,2020-06-24,329200
8,2020-06-23,351800
9,2020-06-22,308300


In [34]:
df_tmp.date === fh5.date

true

Using `view` creates a `SubDataFrame`

In [35]:
dfv_tmp = view(fh5, 1:2, 1:2)

Row,date,volume
,Date,Int64
1,2020-07-02,257500
2,2020-07-01,468100


In [36]:
typeof(dfv_tmp)

SubDataFrame{DataFrame, DataFrames.SubIndex{DataFrames.Index, UnitRange{Int64}, UnitRange{Int64}}, UnitRange{Int64}}

#### Execrcise

Check that `view(fh5, !, :)` and `view(fh5, :, :)` produce the same result

As a warning remember that when you modify the parent of a `SubDataFrame` (or `DataFrameRow`) you may get an error when trying to access it:

In [37]:
df_tmp = fh5[1:3, 1:4]

Row,date,volume,open,high
,Date,Int64,Float64,Float64
1,2020-07-02,257500,17.64,17.74
2,2020-07-01,468100,17.73,17.73
3,2020-06-30,319100,17.65,17.8


In [38]:
dfv_tmp = view(df_tmp, 1:2, 1:3)

Row,date,volume,open
,Date,Int64,Float64
1,2020-07-02,257500,17.64
2,2020-07-01,468100,17.73


In [39]:
select!(df_tmp, 1) # note that in `select` et al. a data frame is always produced

Row,date
,Date
1,2020-07-02
2,2020-07-01
3,2020-06-30


In [40]:
dfv_tmp

BoundsError: BoundsError: attempt to access 1-element Vector{Symbol} at index [1:3]

A special case is when you use `:` as a column selection with a `view`. In this case the `SubDataFrame` and `DataFrameRow` always get updated with the changed columns:

In [41]:
df_tmp = fh5[1:3, 1:4]

Row,date,volume,open,high
,Date,Int64,Float64,Float64
1,2020-07-02,257500,17.64,17.74
2,2020-07-01,468100,17.73,17.73
3,2020-06-30,319100,17.65,17.8


In [42]:
dfv_tmp = view(df_tmp, 1:2, :)

Row,date,volume,open,high
,Date,Int64,Float64,Float64
1,2020-07-02,257500,17.64,17.74
2,2020-07-01,468100,17.73,17.73


In [43]:
select!(df_tmp, 1, :open => :newcol)

Row,date,newcol
,Date,Float64
1,2020-07-02,17.64
2,2020-07-01,17.73
3,2020-06-30,17.65


In [44]:
dfv_tmp

Row,date,newcol
,Date,Float64
1,2020-07-02,17.64
2,2020-07-01,17.73


The reason for this behavior is that subsetting of a data frame by only rows (and taking all columns) is very common, and in this case we can create and index such views much faster. In particular `DataFrameRow`s produced by `eachrow` are efficient this way:

In [45]:
@btime mean(x -> x.open, eachrow(fh5))

  897.894 ms (34259694 allocations: 522.76 MiB)


298.08612911254846

In [46]:
@btime mean(i -> fh5[i, :open], 1:nrow(fh5))

  1.187 s (41111214 allocations: 627.31 MiB)


298.08612911254846

Of course, type-stable operation would be faster (but sometimes processing data row-wise is more convenient):

In [47]:
@btime mean(fh5.open)

  2.289 ms (1 allocation: 16 bytes)


298.08612911254846

or, if your table is not very wide (so that you are not penalized by the compilation cost of `NamedTuple`) you can use:

In [48]:
@btime mean(x -> x.open, Tables.namedtupleiterator(fh5))

  8.837 ms (17 allocations: 992 bytes)


298.0861291125547

<div class="alert alert-block alert-info">
<b>Tip:</b>
    

If your table is not very wide then you can use `Tables.namedtupleiterator` or `Tables.columntable` to swithch a `DataFrame` into a type-stable mode (at the cost of fixing its schema). Both are non-allocating. You can also used `Tables.rowtable` but it is allocating so usually `Tables.namedtupleiterator` is preferred.
</div>

Note though that `DataFrameRow` allows you to modify the source data frame, while iterating `NamedTuple`s is read-only (more on `setindex!` later).

In [49]:
df_tmp = copy(fh5)

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


#### Exercise

In `df_tmp` find rows in which `:high` is less than `:low` and swap these values.
We will discuss three ways to do it.

I give you the following column selectors. Can you tell the effect of each of them when trying to run `fh5[1:2, selector]`?
Write the code that tests it.

In [50]:
selectors = [Between(1, 10), Between(:low, :high), [:low, :low], Cols(:low, :low), Cols(:low, :), All()]

6-element Vector{Any}:
 Between{Int64, Int64}(1, 10)
 Between{Symbol, Symbol}(:low, :high)
 [:low, :low]
 Cols{Tuple{Symbol, Symbol}}((:low, :low), union)
 Cols{Tuple{Symbol, Colon}}((:low, Colon()), union)
 All{Tuple{}}(())

<div class="alert alert-block alert-info">
<b>Tip:</b>
    
In general `df.single_col` and `df[!, single_col]` produce the same result in `getindex`.

The exceptions are:
* `@view` and `@views` does not affect `df.single_col`.
* in `df.single_col` you cannot pass `single_col` as a variable (unless you write `getproperty(df, single_col)`)
* only `df[!, single_col]` allows integer indexing

</div>


### Indexing `GroupedDataFrame`

A `GroupedDataFrame` is a view into a data frame which defines a key allowing a fast lookup (and in particular this key is then automatically used in split-apply-combine operations with `select`, `select!`, `transform`, `transform!` and `combine`).

In [51]:
gdf = groupby(fh5, :symbol)

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [52]:
gdf_keys = keys(gdf)

6335-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (symbol = String7("AAAU"),)
 GroupKey: (symbol = String7("AACG"),)
 GroupKey: (symbol = String7("AADR"),)
 GroupKey: (symbol = String7("AAL"),)
 GroupKey: (symbol = String7("AAMC"),)
 GroupKey: (symbol = String7("AAME"),)
 GroupKey: (symbol = String7("AAN"),)
 GroupKey: (symbol = String7("AAOI"),)
 GroupKey: (symbol = String7("AAON"),)
 GroupKey: (symbol = String7("AAP"),)
 GroupKey: (symbol = String7("AAPL"),)
 GroupKey: (symbol = String7("AAT"),)
 GroupKey: (symbol = String7("AAU"),)
 ⋮
 GroupKey: (symbol = String7("ZOM"),)
 GroupKey: (symbol = String7("ZROZ"),)
 GroupKey: (symbol = String7("ZS"),)
 GroupKey: (symbol = String7("ZSAN"),)
 GroupKey: (symbol = String7("ZTO"),)
 GroupKey: (symbol = String7("ZTR"),)
 GroupKey: (symbol = String7("ZTS"),)
 GroupKey: (symbol = String7("ZUO"),)
 GroupKey: (symbol = String7("ZVO"),)
 GroupKey: (symbol = String7("ZYME"),)
 GroupKey: (symbol = String7("ZYNE"),)
 GroupKey:

As usual - indexing by a single value drops a dimension (you get a `SubDataFrame`)

In [53]:
gdf[1]

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [54]:
gdf_keys[1]

GroupKey: (symbol = String7("AAAU"),)

In [55]:
gdf[gdf_keys[1]]

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [56]:
gdf[(symbol="AAAU",)]

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [57]:
gdf[("AAAU",)]

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


And indexing by a collection produces a subsetted `GroupedDataFrame`:

In [58]:
gdf[1:2]

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [59]:
gdf[tuple.(["AAAU", "AACG"])]

Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


Lookup is backed by `Dict` so it is fast.

In [60]:
@btime $gdf[("AACG",)]

  757.407 ns (10 allocations: 2.89 KiB)


Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,46800,1.3,1.39,1.3,1.31,1.31,AACG
2,2020-07-01,95400,1.25,1.4,1.18,1.31,1.31,AACG
3,2020-06-30,40200,1.16,1.27,1.16,1.26,1.26,AACG
4,2020-06-29,46900,1.15,1.25,1.15,1.17,1.17,AACG
5,2020-06-26,43700,1.12,1.18,1.12,1.15,1.15,AACG
6,2020-06-25,72900,1.22,1.25,1.11,1.23,1.23,AACG
7,2020-06-24,80400,1.14,1.3,1.14,1.25,1.25,AACG
8,2020-06-23,57000,1.19,1.22,1.14,1.17,1.17,AACG
9,2020-06-22,115500,1.0,1.17,0.97,1.14,1.14,AACG


In [61]:
@btime @view $fh5[findall(==("AACG"), $fh5.symbol), :]

  12.982 ms (8 allocations: 843.53 KiB)


Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,46800,1.3,1.39,1.3,1.31,1.31,AACG
2,2020-07-01,95400,1.25,1.4,1.18,1.31,1.31,AACG
3,2020-06-30,40200,1.16,1.27,1.16,1.26,1.26,AACG
4,2020-06-29,46900,1.15,1.25,1.15,1.17,1.17,AACG
5,2020-06-26,43700,1.12,1.18,1.12,1.15,1.15,AACG
6,2020-06-25,72900,1.22,1.25,1.11,1.23,1.23,AACG
7,2020-06-24,80400,1.14,1.3,1.14,1.25,1.25,AACG
8,2020-06-23,57000,1.19,1.22,1.14,1.17,1.17,AACG
9,2020-06-22,115500,1.0,1.17,0.97,1.14,1.14,AACG


In [62]:
@btime filter(:symbol => ==("AACG"), fh5)

  13.055 ms (35 allocations: 863.91 KiB)


Row,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7
1,2020-07-02,46800,1.3,1.39,1.3,1.31,1.31,AACG
2,2020-07-01,95400,1.25,1.4,1.18,1.31,1.31,AACG
3,2020-06-30,40200,1.16,1.27,1.16,1.26,1.26,AACG
4,2020-06-29,46900,1.15,1.25,1.15,1.17,1.17,AACG
5,2020-06-26,43700,1.12,1.18,1.12,1.15,1.15,AACG
6,2020-06-25,72900,1.22,1.25,1.11,1.23,1.23,AACG
7,2020-06-24,80400,1.14,1.3,1.14,1.25,1.25,AACG
8,2020-06-23,57000,1.19,1.22,1.14,1.17,1.17,AACG
9,2020-06-22,115500,1.0,1.17,0.97,1.14,1.14,AACG


<div class="alert alert-block alert-info">
<b>Tip:</b>
    

Think of `GroupedDataFrame` as a wrapper over a data frame object which caches information
about the parent data frame to make operations that rely on row index faster. Currently this is used in:
* split/apply/combine
* lookup

In other words: if you like row indices in Pandas then `GroupedDataFrame` is a way to achieve such functionality in DataFrames.jl.

Notably:
* you can set multiple row indices to the same data frame, just by creating different `GroupedDataFrame` objects on top of the same data frame.
* the creation of cache in grouped data frame is lazy (it is computed only if needed); cache computation is thread safe.

</div>

In [63]:
for i in 1:3
    local gdf
    @show i
    @time gdf = groupby(fh5, :symbol)
    @time gdf[("AACG",)]
    @time gdf[("AACG",)]
    @time gdf[("AACG",)]
end

i = 1
  0.097084 seconds (39 allocations: 232.556 MiB)
  0.035548 seconds (6.39 k allocations: 52.740 MiB)
  0.000011 seconds (12 allocations: 2.953 KiB)
  0.000004 seconds (12 allocations: 2.953 KiB)
i = 2
  0.134641 seconds (39 allocations: 232.556 MiB, 21.04% gc time)
  0.036761 seconds (6.39 k allocations: 52.740 MiB)
  0.000006 seconds (12 allocations: 2.953 KiB)
  0.000010 seconds (12 allocations: 2.953 KiB)
i = 3
  0.130015 seconds (39 allocations: 232.556 MiB)
  0.041583 seconds (6.39 k allocations: 52.740 MiB)
  0.000005 seconds (12 allocations: 2.953 KiB)
  0.000003 seconds (12 allocations: 2.953 KiB)


## setindex!

`setindex!` is defined only for `DataFrame`, `SubDataFrame` and `DataFrameRow` (other types are read-only).

Intended rules:
* the dimensions of left hand side and right hand side must match;
* if right hand side has names they must match the names in left hand side.

special cases (that might be surprising):

In [64]:
x = rand(5, 5)

5×5 Matrix{Float64}:
 0.496863  0.572796  0.387258  0.93069   0.666971
 0.887483  0.576724  0.999825  0.943775  0.279755
 0.809132  0.147583  0.50549   0.734813  0.787697
 0.395642  0.484826  0.612615  0.20297   0.950586
 0.613923  0.112816  0.270383  0.968726  0.0375709

In [65]:
df = DataFrame(x, :auto)

Row,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.496863,0.572796,0.387258,0.93069,0.666971
2,0.887483,0.576724,0.999825,0.943775,0.279755
3,0.809132,0.147583,0.50549,0.734813,0.787697
4,0.395642,0.484826,0.612615,0.20297,0.950586
5,0.613923,0.112816,0.270383,0.968726,0.0375709


In [66]:
x[1, 1:4] = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [67]:
x

5×5 Matrix{Float64}:
 1.0       3.0       2.0       4.0       0.666971
 0.887483  0.576724  0.999825  0.943775  0.279755
 0.809132  0.147583  0.50549   0.734813  0.787697
 0.395642  0.484826  0.612615  0.20297   0.950586
 0.613923  0.112816  0.270383  0.968726  0.0375709

In [68]:
x[1:4, 1] = [1 2; 3 4]

LoadError: DimensionMismatch: tried to assign 2×2 array to 4×1 destination

#### Exercise

Check what happens if you try the same operations on `df`.

<div class="alert alert-block alert-info">
<b>Tip:</b>
    
We want DataFrames.jl to match 100% what Base does with indexing (except for cases where matching column names matters).

If you find cases when it does not please report an issue.

</div>

The most important case, that does work as in Base (as `Matrix` is not resizable) is creation of new columns.

In [69]:
df = DataFrame(rand(2, 2), :auto)

Row,x1,x2
,Float64,Float64
1,0.751146,0.866453
2,0.370838,0.788149


In [70]:
x3 = [1, 2]

2-element Vector{Int64}:
 1
 2

In [71]:
df.x3_1 = x3

2-element Vector{Int64}:
 1
 2

In [72]:
df[:, :x3_2] = x3

2-element Vector{Int64}:
 1
 2

In [73]:
df[!, :x3_3] = x3

2-element Vector{Int64}:
 1
 2

In [74]:
df

Row,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,0.751146,0.866453,1,1,1
2,0.370838,0.788149,2,2,2


#### Exercise
Check in which cases `x3` got copied.

A special case when column always gets copied is for range objects:

In [75]:
fh5.col_idx = axes(fh5, 1)

Base.OneTo(6852038)

In [76]:
fh5.col_idx

6852038-element Vector{Int64}:
       1
       2
       3
       4
       5
       6
       7
       8
       9
      10
      11
      12
      13
       ⋮
 6852027
 6852028
 6852029
 6852030
 6852031
 6852032
 6852033
 6852034
 6852035
 6852036
 6852037
 6852038

Let us try `setindex!` for multi-column indexing.

In [77]:
df

Row,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,0.751146,0.866453,1,1,1
2,0.370838,0.788149,2,2,2


In [78]:
df2 = DataFrame(x1=[10,20], x2=["a","b"])

Row,x1,x2
,Int64,String
1,10,a
2,20,b


In [79]:
df[:, 1:2] = df2

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mFloat64[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mtwiceprecision.jl:273[24m[39m
[0m  convert(::Type{T}, [91m::AbstractChar[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mchar.jl:185[24m[39m
[0m  convert(::Type{T}, [91m::CartesianIndex{1}[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mmultidimensional.jl:127[24m[39m
[0m  ...


Warning: this operation is not atomic currently.

In [80]:
df

Row,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,10.0,0.866453,1,1,1
2,20.0,0.788149,2,2,2


In [81]:
df[!, 1:2] = df2

Row,x1,x2
,Int64,String
1,10,a
2,20,b


In [82]:
df

Row,x1,x2,x3_1,x3_2,x3_3
,Int64,String,Int64,Int64,Int64
1,10,a,1,1,1
2,20,b,2,2,2


<div class="alert alert-block alert-info">
<b>Tip:</b>

In `setindex!` context:
* when `:` is used to select rows it operates in-place (it works just as any row selector); except when adding a new column, in which case it copies a column;
* when `!` is used to select rows it always allocates a new column; if a single column is selected - this is a no-copy operation; if multiple columns are selected a copy is always made
* `df.single_col = v` is exactly the same as `df[!, single_col] = v` if `single_col` is a `Symbol` or string literal
* it is not allowed to add columns to `SubDataFrame` or `DataFrameRow`

</div>

## broadcast

Data frame behaves in broadcasting just like a matrix, with two exceptions:
* it forces the style of the result to be a `DataFrame`
* if several data frames take part in broadcasting they must have matching column names

In [83]:
log.(fh5[!, Between(:open, :close)])

Row,open,high,low,close
,Float64,Float64,Float64,Float64
1,2.87017,2.87582,2.86903,2.87413
2,2.87526,2.87526,2.86448,2.87243
3,2.87074,2.8792,2.86847,2.87807
4,2.87187,2.873,2.8696,2.87243
5,2.86163,2.87187,2.85762,2.87187
6,2.8679,2.8679,2.86334,2.86733
7,2.86847,2.87413,2.86562,2.86847
8,2.86505,2.8713,2.86505,2.8713
9,2.8622,2.86619,2.85877,2.8622


In [84]:
log.(fh5[!, Between(:open, :adjclose)])

LoadError: DomainError with -0.8700000643730164:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [85]:
filter(:adjclose => <=(0), fh5)

Row,date,volume,open,high,low,close,adjclose,symbol,col_idx
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7,Int64
1,2020-05-13,507400,3.4,3.4,2.99,3.02,-0.87,HCHC,2814814
2,2020-05-12,658600,3.22,3.48,3.11,3.19,-0.918974,HCHC,2814815
3,2020-05-11,564700,3.17,3.43,3.12,3.18,-0.916093,HCHC,2814816
4,2020-05-08,264600,2.92,3.08,2.8,3.06,-0.881523,HCHC,2814817
5,2020-05-07,246900,3.06,3.06,2.67,2.8,-0.806623,HCHC,2814818
6,2020-05-06,219500,2.87,3.08,2.78,2.98,-0.858477,HCHC,2814819
7,2020-05-05,190900,2.7,2.84,2.7,2.8,-0.806623,HCHC,2814820
8,2020-05-04,139800,2.58,2.8,2.58,2.69,-0.774934,HCHC,2814821
9,2020-05-01,288800,2.64,2.77,2.54,2.65,-0.763411,HCHC,2814822


In [86]:
describe(fh5)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,date,,2015-01-02,2018-01-17,2020-07-02,0,Date
2,volume,1.01541e6,1,120600.0,2156725200,0,Int64
3,open,298.086,0.001,24.95,6.91553e7,0,Float64
4,high,305.876,0.0,25.11,7.05886e7,0,Float64
5,low,291.014,0.0,24.75,6.84387e7,0,Float64
6,close,296.783,0.001,24.94,6.95136e7,0,Float64
7,adjclose,293.231,-3.77096,23.3258,6.90222e7,0,Float64
8,symbol,,AAAU,,ZYXI,0,String7
9,col_idx,3.42602e6,1,3.42602e6,6852038,0,Int64


#### Exercise

Replace non-positive values in `:adjclose` by a value in `:close` and store it in a variable `:adjclose_fix`.

In [87]:
filter(:adjclose => <=(0), fh5)

Row,date,volume,open,high,low,close,adjclose,symbol,col_idx
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7,Int64
1,2020-05-13,507400,3.4,3.4,2.99,3.02,-0.87,HCHC,2814814
2,2020-05-12,658600,3.22,3.48,3.11,3.19,-0.918974,HCHC,2814815
3,2020-05-11,564700,3.17,3.43,3.12,3.18,-0.916093,HCHC,2814816
4,2020-05-08,264600,2.92,3.08,2.8,3.06,-0.881523,HCHC,2814817
5,2020-05-07,246900,3.06,3.06,2.67,2.8,-0.806623,HCHC,2814818
6,2020-05-06,219500,2.87,3.08,2.78,2.98,-0.858477,HCHC,2814819
7,2020-05-05,190900,2.7,2.84,2.7,2.8,-0.806623,HCHC,2814820
8,2020-05-04,139800,2.58,2.8,2.58,2.69,-0.774934,HCHC,2814821
9,2020-05-01,288800,2.64,2.77,2.54,2.65,-0.763411,HCHC,2814822


and something more advanced now for each stock calculate a column with log-returns of `:close` and store it in `:close_returns`:

In [88]:
df = sort(fh5, :date)

Row,date,volume,open,high,low,close,adjclose,symbol,col_idx
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7,Int64
1,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,AADR,2114
2,2015-01-02,10748600,54.28,54.6,53.07,53.91,51.0799,AAL,3499
3,2015-01-02,11500,308.0,348.59,308.0,327.18,327.18,AAMC,4798
4,2015-01-02,11400,3.99,4.03,3.98,4.03,3.91772,AAME,6079
5,2015-01-02,898900,30.81,30.86,30.04,30.62,30.0588,AAN,7464
6,2015-01-02,184600,11.28,11.28,10.72,10.79,10.79,AAOI,8840
7,2015-01-02,90700,22.55,22.68,21.6,21.93,20.9958,AAON,10225
8,2015-01-02,509800,160.85,162.5,157.47,158.56,156.251,AAP,11610
9,2015-01-02,53204600,111.39,111.44,107.35,109.33,99.9459,AAPL,12995


In [89]:
gdf = groupby(df, :symbol)

Row,date,volume,open,high,low,close,adjclose,symbol,col_idx
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7,Int64
1,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,AADR,2114
2,2015-01-05,600,36.13,36.44,36.13,36.19,34.965,AADR,2113
3,2015-01-06,600,35.57,35.89,35.56,35.89,34.6752,AADR,2112
4,2015-01-08,1100,36.73,36.8,36.45,36.45,35.2162,AADR,2111
5,2015-01-12,100,36.39,36.39,36.39,36.39,35.1582,AADR,2110
6,2015-01-14,1100,36.2,36.25,36.2,36.25,35.023,AADR,2109
7,2015-01-16,200,36.25,36.25,36.25,36.25,35.023,AADR,2108
8,2015-01-20,2000,36.7,36.7,36.47,36.5,35.2645,AADR,2107
9,2015-01-22,1700,36.75,37.23,36.75,37.23,35.9698,AADR,2106


In [90]:
df2 = transform(gdf, :close => (x -> log.(x ./ lag(x))) => :close_returns)

Row,date,volume,open,high,low,close,adjclose,symbol,col_idx,close_returns
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String7,Int64,Float64?
1,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,AADR,2114,missing
2,2015-01-02,10748600,54.28,54.6,53.07,53.91,51.0799,AAL,3499,missing
3,2015-01-02,11500,308.0,348.59,308.0,327.18,327.18,AAMC,4798,missing
4,2015-01-02,11400,3.99,4.03,3.98,4.03,3.91772,AAME,6079,missing
5,2015-01-02,898900,30.81,30.86,30.04,30.62,30.0588,AAN,7464,missing
6,2015-01-02,184600,11.28,11.28,10.72,10.79,10.79,AAOI,8840,missing
7,2015-01-02,90700,22.55,22.68,21.6,21.93,20.9958,AAON,10225,missing
8,2015-01-02,509800,160.85,162.5,157.47,158.56,156.251,AAP,11610,missing
9,2015-01-02,53204600,111.39,111.44,107.35,109.33,99.9459,AAPL,12995,missing


In [91]:
combine(sdf -> first(sdf, 3), groupby(df2, :symbol), ungroup=false)

Row,symbol,date,volume,open,high,low,close,adjclose,col_idx,close_returns
,String7,Date,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Float64?
1,AADR,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,2114,missing
2,AADR,2015-01-05,600,36.13,36.44,36.13,36.19,34.965,2113,-0.0123577
3,AADR,2015-01-06,600,35.57,35.89,35.56,35.89,34.6752,2112,-0.00832411
Row,symbol,date,volume,open,high,low,close,adjclose,col_idx,close_returns
,String7,Date,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Float64?
1,EAI,2020-07-02,10150,25.29,25.48,25.37,25.4042,25.4042,1747793,missing


#### Exercise

Check our codes to make sure that in each group in `df2` first and only the fist element of `:close_returns` is `missing`

#### Exercises

Check what happens if you try to broadcast a sum of a 1-row `DataFrame` with an array having multiple rows

Check what happens if you try to broadcast a sum of a 0-row `DataFrame` with an array having multiple rows

Check what happens when you try to broadcast a sum of a `DataFrame` with a 3D array.

Check if broadcasting is defined for `DataFrameRow`.

<div class="alert alert-block alert-info">
<b>Tip:</b>

In broadcasting `df.single_col` and `df[!, single_col]` behave in the same way (the same exceptions as in `getindex` apply).

</div>

## broadcast!

It is possible to assign a value to `AbstractDataFrame` and `DataFrameRow` objects using the `.=` operator.

In such an operation `AbstractDataFrame` is considered as two-dimensional and `DataFrameRow` as single-dimensional (columnar).

#### Special cases:

Broadcasting into a single cell unwraps it before opertion:

In [92]:
df = DataFrame(a=[[1,2], [3,4]])

Row,a
,Array…
1,"[1, 2]"
2,"[3, 4]"


In [93]:
df[1,1] .= [10, 20]

2-element Vector{Int64}:
 10
 20

In [94]:
df

Row,a
,Array…
1,"[10, 20]"
2,"[3, 4]"


Broadcasting into any slice of a data frame is in-place except for two cases:
* `!` is used as a row selector
* `:` is used as a row selector and column selector is a single column that does not exist in a data frame
* `getproperty` is used as column selector (valid since Julia 1.7!)

In these cases a new column is allocated.

<div class="alert alert-block alert-info">
<b>Tip:</b>

These rules mean that `df.single_col .= v` behaves in the same way as `df[!, single_col] .= v`

(this is valid since Julia 1.7).
</div>

## Summary

All the rules how indexing works in DataFrames.jl are specified here:

https://juliadata.github.io/DataFrames.jl/latest/lib/indexing/

If you find some cases where the behavior does not match these rules please report an issue.

<div class="alert alert-block alert-info">
<b>Tip:</b>

Rules for most common operations:
* get a column without copying `df.single_col` or `df[!, single_col]`
* get a column with copying `df[:, single_col]`


* assign a column without copying `df.single_col = vector` or `df[!, single_col] = vector` (except for ranges)
* assign a column with copying `df[:, single_col] = vector`
* assign a value in-place with broadcasting if column exists:  `df.single_col .= value` or `df[:, single_col] .= value`
* assign a value with copying, create a column if it does not exist: `df[!, single_col] .= value`
* for convenience also `df[:, single_col] .= value` creates a column with copying if it does not exist

</div>